In [ ]:
#from pyspark.sql.functions import unbase64
from pyspark.sql.types import *
from pyspark.sql.functions import *
# Source with default settings
from pyspark.sql import functions as F
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vectors
from pyspark.ml.classification import LogisticRegression

# Prepare training data from a list of (label, features) tuples.
training = spark.createDataFrame([
    (1.0, Vectors.dense([50.0, 20.0, 20.0])),
    (0.0, Vectors.dense([60.0, 30.0, 30.0])),
    (0.0, Vectors.dense([70, 25.0, 250.0])),
    (1.0, Vectors.dense([65, 18.0, 18.0]))], ["label", "features"])

# Create a LogisticRegression instance. This instance is an Estimator.
lr = LogisticRegression(maxIter=10, regParam=0.01)
# Print out the parameters, documentation, and any default values.
print("LogisticRegression parameters:\n" + lr.explainParams() + "\n")

# Learn a LogisticRegression model. This uses the parameters stored in lr.
model1 = lr.fit(training)

# Since model1 is a Model (i.e., a transformer produced by an Estimator),
# we can view the parameters it used during fit().
# This prints the parameter (name: value) pairs, where names are unique IDs for this
# LogisticRegression instance.
print("Model 1 was fit using parameters: ")
print(model1.extractParamMap())

# We may alternatively specify parameters using a Python dictionary as a paramMap
paramMap = {lr.maxIter: 20}
paramMap[lr.maxIter] = 30  # Specify 1 Param, overwriting the original maxIter.
paramMap.update({lr.regParam: 0.1, lr.threshold: 0.55})  # Specify multiple Params.

# You can combine paramMaps, which are python dictionaries.
paramMap2 = {lr.probabilityCol: "myProbability"}  # Change output column name
paramMapCombined = paramMap.copy()
paramMapCombined.update(paramMap2)

# Now learn a new model using the paramMapCombined parameters.
# paramMapCombined overrides all parameters set earlier via lr.set* methods.
model2 = lr.fit(training, paramMapCombined)
print("Model 2 was fit using parameters: ")
print(model2.extractParamMap())



#connectionString = "Endpoint=sb://iothub-ns-ks-bigdata-5809426-7048d14abf.servicebus.windows.net/;SharedAccessKeyName=iothubowner;SharedAccessKey=sQQPHOQhXdvNxTBcBjTfz2fWOTTTnRwRZu+/9HKbXmM=;EntityPath=ks-bigdata-iot-hub-test"
connectionString = "Endpoint=sb://iothub-ns-ks-bigdata-5809426-7048d14abf.servicebus.windows.net/;SharedAccessKeyName=iothubowner;SharedAccessKey=sQQPHOQhXdvNxTBcBjTfz2fWOTTTnRwRZu+/9HKbXmM=;EntityPath=ks-bigdata-iot-hub-test"


ehConf = {
  'eventhubs.connectionString' : connectionString
}

ehConf['eventhubs.connectionString'] = sc._jvm.org.apache.spark.eventhubs.EventHubsUtils.encrypt(connectionString)

# Schema of incoming data from IoT hub
schema = "timestamp timestamp, temperature double, humidity double, deviceId string, status string"

# Read directly from IoT Hub using the EventHubs library for Databricks
iot_stream = (
  spark.readStream.format("eventhubs")                                              # Read from IoT Hubs directly
    .options(**ehConf)                                                              # Use the Event-Hub-enabled connect 
    .load()                                                                         # Load the data
    .withColumn('reading', F.from_json(F.col('body').cast('string'), schema))       # Extract the "body" payload from  
)

#display(iot_stream)

iot_stream = iot_stream.withColumn('temperature', iot_stream.reading.temperature).withColumn('humidity', iot_stream.reading.humidity).withColumn('deviceId', iot_stream.reading.deviceId).withColumn('ts', iot_stream.reading.timestamp)
iot_stream2 = iot_stream.groupBy(window(iot_stream.ts, "5 seconds", "3 seconds"), iot_stream.deviceId).agg(avg('humidity'), avg('temperature'), min('humidity'), min('temperature'), max('humidity'), max('temperature'))

assembler = VectorAssembler(
    inputCols=["avg(humidity)", 'avg(temperature)', 'avg(temperature)'],
    outputCol="features")

features = assembler.transform(iot_stream2.withColumn('tsp', iot_stream2.window.start))
#print("Assembled columns 'hour', 'mobile', 'userFeatures' to vector column 'features'")
#output.select("features", "clicked").show(truncate=False)

predictions = model1.transform(features)
predictions.printSchema
display(predictions)

